# The QUAD method

Click here for an interactive version of this notebook:-
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/pjohno/MSc-Math-Finance-2018/master?filepath=notebooks%2FMSc%20Project%207%20-%20Exotic%20Options.ipynb)


Consider the well-known Black and Scholes (1973) partial differential equation for an option with an underlying
asset following geometric Brownian motion:
$$
\frac{\partial{V}}{\partial{t}} + \frac{1}{2} {\sigma}^2 S^2
\frac{\partial {^2V}}{\partial{S^2}} + (r-D_c) S
\frac{\partial{V}}{\partial{S}} - rV = 0,
$$
where $V(S,t)$ is the price of the derivative product, $S$ the
current value of the underlying asset, $t$ is time,  $T$ is the time to maturity
 $r$ the risk-free interest rate, $\sigma$ the
volatility of the underlying asset and $X$ is the exercise price of the option. 
$D_c$ is a continuous dividend yield which could, for example, be the foreign 
interest rate in a foreign exchange option.

Next make the following standard
transformations                
$$
x = log(S_0),
$$
$$
y = log(S_{T}).
$$
The final conditions for a European option expiring at time $T$
with  $V(y,T)$ are transformed in straightforward fashion; e.g. 
the payoff for a call option $V(S,T)  = \max(S_{T}-X,0)$ becomes
$$
\quad V(y,T) = \max(e^y-X,0). 
$$
The value of 
the option at time $t=0$ on an underlying asset  $S_0$  
has an  exact solution given by 
$$
V(x,0) = A (x) \int_{-\infty}^{\infty} B(x,y)V(y,T) \, dy,
$$
where, 
$$
A(x) = \frac{1}{\sqrt{2\sigma^2\pi T}} e^{-\frac12 kx-\frac18 
\sigma^2k^2 T - r T},
$$
and
$$
B(x,y) = e^{-\frac{(x-y)^2}{2\sigma^2 T} + \frac12 ky}
$$
and
$$
k = \frac{2(r-D_c)}{{\sigma}^2} -1.
$$

We solve this problem by setting up a class structure containing all the relavent functions, which we will call QUAD. This will contain the functions $A$, $B$, a method for integrating and a method to return the value $V(x,0)$. Since integrations must work on fixed spaced grids, we also provide a function to do this. First include libraries:

In [1]:
#include "msc_project_7.hpp"

Now include a structure to hold the functions. We use the C++11 functional library to allow the payoff of the function to be specified by the user. This means the function can be anything we like. Numerically we also will have to bound the intergration, so we need a way to input these values. In fact the integration will look like
$$
V(x,0) = A (x) \int_\text{yMin}^\text{yMax} B(x,y)V(y,T) \, dy,
$$
and by using this approximation we are assuming that the payoff is zero for $y<\text{yMin}$ and $y>\text{yMax}$.

Input the function $A$ as given by the formula:

In [2]:
// the function A
double A(double x, double r, double sigma, double k, double dt)
{
    return 1. / sqrt(2 * sigma*sigma*M_PI*dt) * exp(-0.5*k*x - 0.125*sigma*sigma*k*k*dt - r*dt);
}

Input the function $B$ as given by the formula:

In [3]:
// the function B
double B(double x, double y, double sigma, double k, double dt)
{
    return exp(-(x - y)*(x - y) / (2.*sigma*sigma*dt) + 0.5 * k * y);
}

Here we include a method for integrating a function. We allow the region of integration to be restricted if required for optimisation.

In [4]:
// integration proceedure: integrate the function f_i at the points y_i, contained in a vector integrand
// PLEASE note that here we are assuming that the grid is equally spaced
// nDown, nUp reference the subsection of the vector y we wish to integrate over
double integrate(int nDown, int nUp,const std::vector<double> &y,const std::vector<double> &integrand)
{
    double sum, h = (y[nUp] - y[nDown]) / (nUp - nDown);
    sum = integrand[nDown];
    for (int i = nDown+2; i<nUp; i+=2)
    {
        sum += 2.*integrand[i];
    }
    for (int i = nDown+1; i<nUp; i+=2)
    {
        sum +=  4.*integrand[i];
    }
    sum += integrand[nUp];
    return h / 3.*sum;
}

Resetting the grid over an interval:

In [5]:
// reset, resize and generate new grid with n+1 nodes, equally spaced on the range [min,max]
void resetGrid(int n, double min, double max, std::vector<double> &x)
{
    x.resize(n + 1);
    double dx = (max - min) / double(n);
    if (n == 0)dx = 0.;
    for (uint i = 0; i<x.size(); i++) x[i] = min + i*dx;
}

Now input the value function. We first set up the grid over the interval by applying a log transform
$$
x = \log(S_0)
$$
and
$$
y \in [ \log(S_\text{Min}) , \log (S_\text{max})] .
$$
After the variable $k$ is assigned, we can generate values of the integrand, $I_j$
$$
I_j = B(x,y_j) V(y_j,T) = B(x,y_j) f( e^{y_j} )
$$
where $f$ is the payoff function.

Then the value of the option can be found as
$$
V(x,0) = A(x) \frac{\text{yMax}-\text{yMin}}{3 n}\left( I_0 + 2\sum I_{2j} + 4\sum I_{2j+1}+ I_n \right)
$$

In [6]:
double valueOption(double  S0,double r,double sigma,double T,int n,
                   const std::vector<double> &y,const std::vector<double> &payoff_vec)
{
    // value of x
    double x = log(S0);
    
    // variable k
    double k = 2.*r / sigma / sigma - 1.;
           
    // setup vectors to store option values, and integrand function
    // these vectors must be the same size as the y vector
    std::vector<double> integrand(y.size());
    
    for (uint j = 0; j < y.size(); j++)
    {
        integrand[j] = B( x , y[j] , sigma , k , T ) * payoff_vec[j];
    }
    // in this example we integrate over the entire range of y, it is possible to select a subrange
    return A(x, r, sigma, k, T )*integrate(0, n, y, integrand);
      
}   

Insert some variable for parameters:

In [7]:
double  S,X,r,sigma,T;

Now calculate the value of a call option. To do this we set up two vectors `y` and `V` to store the grid and payoff at the terminal time.

In [8]:
S = 100.; X = 100.; r = 0.06; sigma = 0.2; T = 1.;

{
    int N=500;
    std::cout << " A call option ...\n";
    // set y grid up at the final step, using strike price as lower limit
    std::vector<double> y(N+1),V(N+1);
    resetGrid(N,log(X),log(X*exp(7.5*sigma*sqrt(T))),y);
    // assign payoff, since e^y>=X we don't need max function
    for(unsigned int i=0;i<y.size();i++)
        V[i] = exp(y[i])-X;

    // return the value V(x=log(S),0) = A(x)*\int B(x,y)*V(y,T) dy
    double value = valueOption(S,r,sigma,T,N,y,V);
    std::cout << "C("<<S<<") = " << value << "\n";
}

 A call option ...
C(100) = 10.9895


In [9]:
{
    int N=500;
    std::cout << " A put option ...\n";
    // set y grid up at the final step, using strike price as lower limit
    std::vector<double> y(N+1),V(N+1);
    resetGrid(N,log(X*exp(-7.5*sigma*sqrt(T))),log(X),y);
    // assign payoff, since e^y>=X we don't need max function
    for(unsigned int i=0;i<y.size();i++)
        V[i] = X - exp(y[i]);
    double value = valueOption(S,r,sigma,T,N,y,V);
    std::cout << "P("<<S<<") = "<< value << "\n";
}

 A put option ...
P(100) = 5.166


In [10]:
double callOption(double  S,double X,double r,double sigma,double T,int N=500,double xi=7.5)
{
    // set y grid up at the final step, using strike price as lower limit
    std::vector<double> y(N+1),V(N+1);
    resetGrid(N,log(X),log(X*exp(xi*sigma*sqrt(T))),y);
    // assign payoff, since e^y>=X we don't need max function
    for(unsigned int i=0;i<y.size();i++)
        V[i] = exp(y[i])-X;
    
    // return the value V(x=log(S),0) = A(x)*\int B(x,y)*V(y,T) dy
    return valueOption(S,r,sigma,T,N,y,V);
}

In [11]:
std::cout << " Examine convergence of a call option using QUAD...\n\n";
tableRow("N","V(S,0;N)","R","conv rate");
emptyTableRow(4);
double valueOld = 1.,diffOld=1.;
int k=2;
for(int n=10;n<=5000;n*=k)
{
    double value = callOption(S,X,r,sigma,T,n);
    double diff = value - valueOld;
    double R = diffOld/diff;
    tableRow( n ,value ,R ,log(R)/log(k));
    valueOld = value;
    diffOld = diff;
}

 Examine convergence of a call option using QUAD...

|             N|      V(S,0;N)|             R|     conv rate|
|--------------|--------------|--------------|--------------|
|            10|       11.0476|     0.0995259|      -3.32878|
|            20|       10.9919|      -180.377|          -nan|
|            40|       10.9897|       24.8224|       4.63357|
|            80|       10.9896|       17.0552|       4.09214|
|           160|       10.9895|       16.2434|       4.02178|
|           320|       10.9895|       16.0597|       4.00538|
|           640|       10.9895|       16.0149|       4.00134|
|          1280|       10.9895|       16.0037|       4.00033|
|          2560|       10.9895|        16.001|       4.00009|


See the full code solution on github [click here](https://github.com/pjohno/MSc-Math-Finance-2018/blob/master/main/project-7-exotic-options.cpp).